In [34]:
import pandas as pd
import numpy as np
import math

from surprise import Reader, Dataset
from surprise import SVD, evaluate
from surprise import NMF

In [35]:
df_sub = pd.read_csv('./train_submissions.csv')
df_prob = pd.read_csv('./problem_data.csv')
df_user = pd.read_csv('./user_data.csv')
test= pd.read_csv('./test_submissions_NeDLEvX.csv')


In [36]:
test.shape

(66555, 3)

In [37]:
df_sub.head()

,user_id,problem_id,attempts_range
0,user_232,prob_6507,1
1,user_3568,prob_2994,3
2,user_1600,prob_5071,1
3,user_2256,prob_703,1
4,user_2321,prob_356,1


In [38]:
df_user.head()

,user_id,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds
0,user_3311,47,40,0,NaN,4,1504111645,348.337,330.849,intermediate,1466686436
1,user_3028,63,52,0,India,17,1498998165,405.677,339.450,intermediate,1441893325
2,user_2268,226,203,-8,Egypt,24,1505566052,307.339,284.404,beginner,1454267603
3,user_480,611,490,1,Ukraine,94,1505257499,525.803,471.330,advanced,1350720417
4,user_650,504,479,12,Russia,4,1496613433,548.739,486.525,advanced,1395560498


In [39]:
df_prob.head()

,problem_id,level_type,points,tags
0,prob_3649,H,NaN,NaN
1,prob_6191,A,NaN,NaN
2,prob_2020,F,NaN,NaN
3,prob_313,A,500.0,"greedy,implementation"
4,prob_101,A,500.0,"constructive algorithms,greedy,math"


In [40]:
test.head()

,ID,user_id,problem_id
0,user_856_prob_5822,user_856,prob_5822
1,user_2642_prob_2334,user_2642,prob_2334
2,user_2557_prob_2920,user_2557,prob_2920
3,user_1572_prob_4598,user_1572,prob_4598
4,user_295_prob_6139,user_295,prob_6139


In [41]:
df_sub['user_id'].nunique()

3529

In [42]:
df_sub['problem_id'].nunique()

5776

In [43]:
df_sub.head()

,user_id,problem_id,attempts_range
0,user_232,prob_6507,1
1,user_3568,prob_2994,3
2,user_1600,prob_5071,1
3,user_2256,prob_703,1
4,user_2321,prob_356,1


In [45]:
reader = Reader(rating_scale=(1.0, 6.0))


In [46]:
data = Dataset.load_from_df(df_sub[['user_id', 'problem_id', 'attempts_range']], reader)

In [47]:
data.split(n_folds=5) # 5-fold cross validation

In [48]:
algo = SVD(10) # matrix factorization using 10 latent factors
evaluate(algo, data, measures=['MAE']) # check validation scores

/anaconda3/lib/python3.7/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/anaconda3/lib/python3.7/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating MAE of algorithm SVD.

------------
Fold 1
MAE:  0.6806
------------
Fold 2
MAE:  0.6865
------------
Fold 3
MAE:  0.6833
------------
Fold 4
MAE:  0.6921
------------
Fold 5
MAE:  0.6883
------------
------------
Mean MAE : 0.6862
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.6805682378494718,
                             0.6865331854978132,
                             0.683268162068058,
                             0.6920934346589396,
                             0.6883068122235662]})

In [49]:
trainset = data.build_full_trainset() # train on full training set
algo.train(trainset)

/anaconda3/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [50]:
collab_pred = test.iloc[:,1:].apply(lambda x: algo.predict(x[0],x[1]),axis=1)

In [51]:
final_pred = pd.DataFrame([math.floor(i[3]) for i in collab_pred])

In [52]:
final_pred.loc[:,0].value_counts()

1    48770
2    16309
3     1387
4       87
5        2
Name: 0, dtype: int64

In [33]:
final_pred.to_csv('collab_filter.csv')